In [1]:
! pip install seqeval transformers torch datasets transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import Dataset
import numpy as np
from transformers import EvalPrediction
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load your DataFrame
df = pd.read_csv('news_price.csv')
# Drop rows where 'summary', 'subject', or 'action' are NaN
df = df.dropna(subset=['summary', 'subject', 'action'])

# Combine 'title' and 'summary' into a single text column
df['text'] = df['title'] + " " + df['summary']

# Map your labels to integers if they are not already
label_dict = {'long': 0, 'short': 1}
df['label'] = df['action'].map(label_dict)

# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert DataFrame to Hugging Face's Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Load pre-trained tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Tokenize the input (text)
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

# Define training arguments
training_args = TrainingArguments(
    output_dir='.',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='.',
    load_best_model_at_end=True,
    save_strategy="steps",  # or "epoch" depending on your preference
    evaluation_strategy="steps",  # should match save_strategy
    logging_steps=50,  # How often to log loss; adjust to your preference
    eval_steps=50,  # How often to run evaluation; adjust to your preference
)


def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Define the trainer with the compute_metrics function
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)


# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1184 [00:00<?, ? examples/s]

Map:   0%|          | 0/297 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,0.699200,0.683696,0.569024,0.280899,0.471698,0.200000
100,0.692500,0.695188,0.491582,0.501650,0.426966,0.608000
150,0.675100,0.685537,0.545455,0.498141,0.465278,0.536000
200,0.689500,0.686619,0.602694,0.119403,0.888889,0.064000
250,0.689100,0.685940,0.585859,0.517647,0.507692,0.528000
300,0.679800,0.690361,0.602694,0.106061,1.000000,0.056000
350,0.653400,0.689646,0.599327,0.304094,0.565217,0.208000
400,0.653500,0.666512,0.606061,0.445498,0.546512,0.376000


{'eval_loss': 0.7074891328811646, 'eval_accuracy': 0.5824915824915825, 'eval_f1': 0.1388888888888889, 'eval_precision': 0.5263157894736842, 'eval_recall': 0.08, 'eval_runtime': 330.8188, 'eval_samples_per_second': 0.898, 'eval_steps_per_second': 0.057, 'epoch': 3.0}


{'eval_loss': 0.6935545802116394,
 'eval_runtime': 428.3735,
 'eval_samples_per_second': 0.693,
 'eval_steps_per_second': 0.044,
 'epoch': 3.0}